# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)


#compare TF and Count performance 


X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,3.476181,0.066410,0.161418,0.015710,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.977558,0.973968,0.975741,0.973944,0.971249,0.974492,0.002100,1
10,26.496631,0.392657,0.284767,0.011894,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978456,0.976661,0.972147,0.966757,0.973944,0.973593,0.004050,2
11,48.159526,3.805138,0.353217,0.046195,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.977558,0.973046,0.968553,0.970350,0.973593,0.003888,3
8,46.810333,0.257147,0.376392,0.010551,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.976661,0.973944,0.968553,0.970350,0.973413,0.003493,4
7,24.482869,0.155278,0.256744,0.007219,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979354,0.975763,0.973046,0.967655,0.971249,0.973413,0.003970,5


In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,3.391627,0.046775,0.153086,0.011692,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.976661,0.974865,0.975741,0.965858,0.971249,0.972875,0.003960,1
7,24.407778,0.092499,0.259552,0.012327,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.974865,0.973944,0.966757,0.971249,0.972875,0.003664,2
8,46.637603,0.162743,0.371789,0.006249,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.975763,0.974843,0.968553,0.968553,0.972875,0.003575,2
11,46.312079,4.679729,0.356397,0.046930,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.973070,0.974843,0.968553,0.969452,0.972516,0.003098,4
10,26.369345,0.228023,0.297650,0.013867,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.974865,0.973944,0.969452,0.966757,0.972156,0.003465,5
